<a href="https://colab.research.google.com/github/anandgokul18/Tiny-ImageNet-Recognition/blob/master/TinyImageNet_Model_Classification_42%25_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!pip install pydrive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
    100% |████████████████████████████████| 993kB 18.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built pydrive
Mounted at /content/gdrive


In [0]:
## Download the dataset and unzip it everytime the session crashes.
#64x64 Images
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

--2019-04-04 17:53:44--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  20.4MB/s    in 15s     

2019-04-04 17:54:00 (16.2 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [0]:
## Download the dataset and unzip it everytime the session crashes.
#32x32 Images
'''
!curl -c /tmp/cookies "https://drive.google.com/uc?export=download&id=1rkKhiSCqlcI3DwxGOJr-uyKVIvZdBcXp" > /tmp/intermezzo.html
! curl -L -b /tmp/cookies "https://drive.google.com$(cat /tmp/intermezzo.html | grep -Po 'uc-download-link" [^>]* href="\K[^"]*' | sed 's/\&amp;/\&/g')" > compressed32.zip
'''

'\n!curl -c /tmp/cookies "https://drive.google.com/uc?export=download&id=1rkKhiSCqlcI3DwxGOJr-uyKVIvZdBcXp" > /tmp/intermezzo.html\n! curl -L -b /tmp/cookies "https://drive.google.com$(cat /tmp/intermezzo.html | grep -Po \'uc-download-link" [^>]* href="\\K[^"]*\' | sed \'s/\\&amp;/\\&/g\')" > compressed32.zip\n'

In [0]:
## Skip the unzip if unzipped dataset is already present in the directory.

!unzip -qq 'tiny-imagenet-200.zip'
!ls
!rm -rf tiny-imagenet-200.zip
!ls


drive  gdrive  sample_data  tiny-imagenet-200  tiny-imagenet-200.zip
drive  gdrive  sample_data  tiny-imagenet-200


In [0]:
#32x32 images
'''
## Skip the unzip if unzipped dataset is already present in the directory.
!unzip -qq 'tiny-imagenet-200.zip'
!unzip -qq 'compressed32.zip'
!ls
!rm -rf tiny-imagenet-200.zip
!rm -rf compressed32.zip
!rm -rf ./tiny-imagenet-200/train/.DS_Store
!rm -rf _MACOSX
!rm -rf compressed32
'''

"\n## Skip the unzip if unzipped dataset is already present in the directory.\n!unzip -qq 'tiny-imagenet-200.zip'\n!unzip -qq 'compressed32.zip'\n!ls\n!rm -rf tiny-imagenet-200.zip\n!rm -rf compressed32.zip\n!rm -rf ./tiny-imagenet-200/train/.DS_Store\n!rm -rf _MACOSX\n!rm -rf compressed32\n"

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [0]:

tf.test.is_gpu_available()


True

In [0]:
tf.test.gpu_device_name()


'/device:GPU:0'

In [0]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 633562715712725013, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10226405047292941002
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7474089316964742285
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11276946637
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14130686733779491162
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib  inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
'''


THE BELOW BLOCK CONTAINS LOADING THE TRAIN AND VALIDATION DATSETS


'''

'\n\n\nTHE BELOW BLOCK CONTAINS LOADING THE TRAIN AND VALIDATION DATSETS\n\n\n'

In [0]:
#LOADING THE IMAGENET INTO x_train and x_test...

#This is obtained from https://github.com/miquelmarti/tiny-imagenet-classifier/blob/master/load_images.py
#since our focus is to develop insight into Architecture not loading data.


# load the val annotations file

import os

def get_annotations_map():
	valAnnotationsPath = './tiny-imagenet-200/val/val_annotations.txt'
	valAnnotationsFile = open(valAnnotationsPath, 'r')
	valAnnotationsContents = valAnnotationsFile.read()
	valAnnotations = {}

	for line in valAnnotationsContents.splitlines():
		pieces = line.strip().split()
		valAnnotations[pieces[0]] = pieces[1]

	return valAnnotations

import numpy as np
from PIL import Image

def load_images(path,num_classes):
    #Load images
    
    print('Loading ' + str(num_classes) + ' classes')

    X_train=np.zeros([num_classes*500,3,64,64],dtype='uint8')
    y_train=np.zeros([num_classes*500], dtype='uint8')

    trainPath=path+'/train'

    print('loading training images...');

    i=0
    j=0
    annotations={}
    for sChild in os.listdir(trainPath):
        sChildPath = os.path.join(os.path.join(trainPath,sChild),'images')
        annotations[sChild]=j
        for c in os.listdir(sChildPath):
            X=np.array(Image.open(os.path.join(sChildPath,c)))
            if len(np.shape(X))==2:
                X_train[i]=np.array([X,X,X])
            else:
                X_train[i]=np.transpose(X,(2,0,1))
            y_train[i]=j
            i+=1
        j+=1
        if (j >= num_classes):
            break

    print('finished loading training images')

    val_annotations_map = get_annotations_map()

    X_test = np.zeros([num_classes*50,3,64,64],dtype='uint8')
    y_test = np.zeros([num_classes*50], dtype='uint8')


    print('loading test images...')

    i = 0
    testPath=path+'/val/images'
    for sChild in os.listdir(testPath):
        if val_annotations_map[sChild] in annotations.keys():
            sChildPath = os.path.join(testPath, sChild)
            X=np.array(Image.open(sChildPath))
            if len(np.shape(X))==2:
                X_test[i]=np.array([X,X,X])
            else:
                X_test[i]=np.transpose(X,(2,0,1))
            y_test[i]=annotations[val_annotations_map[sChild]]
            i+=1
        else:
            pass


    print('finished loading test images'+str(i))

    return X_train,y_train,X_test,y_test
  
path='./tiny-imagenet-200'
num_classes = 200
X_train,y_train,X_test,y_test=load_images(path,num_classes)


print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

num_samples=len(X_train)

Loading 200 classes
loading training images...
finished loading training images
loading test images...
finished loading test images10000
X_train shape: (100000, 3, 64, 64)
100000 train samples
10000 test samples


In [0]:
#LOADING THE IMAGENET INTO x_train and x_test...
#32x32 images
'''
#This is obtained from https://github.com/miquelmarti/tiny-imagenet-classifier/blob/master/load_images.py
#since our focus is to develop insight into Architecture not loading data.


# load the val annotations file

import os

def get_annotations_map():
	valAnnotationsPath = './tiny-imagenet-200/val/val_annotations.txt'
	valAnnotationsFile = open(valAnnotationsPath, 'r')
	valAnnotationsContents = valAnnotationsFile.read()
	valAnnotations = {}

	for line in valAnnotationsContents.splitlines():
		pieces = line.strip().split()
		valAnnotations[pieces[0]] = pieces[1]

	return valAnnotations

import numpy as np
from PIL import Image

def load_images(path,num_classes):
    #Load images
    
    print('Loading ' + str(num_classes) + ' classes')

    X_train=np.zeros([num_classes*500,3,32,32],dtype='uint8')
    y_train=np.zeros([num_classes*500], dtype='uint8')

    trainPath=path+'/train'

    print('loading training images...');

    i=0
    j=0
    annotations={}
    for sChild in os.listdir(trainPath):
        sChildPath = os.path.join(os.path.join(trainPath,sChild),'images')
        annotations[sChild]=j
        for c in os.listdir(sChildPath):
            X=np.array(Image.open(os.path.join(sChildPath,c)))
            if len(np.shape(X))==2:
                X_train[i]=np.array([X,X,X])
            else:
                X_train[i]=np.transpose(X,(2,0,1))
            y_train[i]=j
            i+=1
        j+=1
        if (j >= num_classes):
            break

    print('finished loading training images')

    val_annotations_map = get_annotations_map()

    X_test = np.zeros([num_classes*50,3,64,64],dtype='uint8')
    y_test = np.zeros([num_classes*50], dtype='uint8')


    print('loading test images...')

    i = 0
    testPath=path+'/val/images'
    for sChild in os.listdir(testPath):
        if val_annotations_map[sChild] in annotations.keys():
            sChildPath = os.path.join(testPath, sChild)
            X=np.array(Image.open(sChildPath))
            if len(np.shape(X))==2:
                X_test[i]=np.array([X,X,X])
            else:
                X_test[i]=np.transpose(X,(2,0,1))
            y_test[i]=annotations[val_annotations_map[sChild]]
            i+=1
        else:
            pass


    print('finished loading test images'+str(i))

    return X_train,y_train,X_test,y_test
  
path='./tiny-imagenet-200'
num_classes = 200
X_train,y_train,X_test,y_test=load_images(path,num_classes)


print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

num_samples=len(X_train)'''

"\n#This is obtained from https://github.com/miquelmarti/tiny-imagenet-classifier/blob/master/load_images.py\n#since our focus is to develop insight into Architecture not loading data.\n\n\n# load the val annotations file\n\nimport os\n\ndef get_annotations_map():\n\tvalAnnotationsPath = './tiny-imagenet-200/val/val_annotations.txt'\n\tvalAnnotationsFile = open(valAnnotationsPath, 'r')\n\tvalAnnotationsContents = valAnnotationsFile.read()\n\tvalAnnotations = {}\n\n\tfor line in valAnnotationsContents.splitlines():\n\t\tpieces = line.strip().split()\n\t\tvalAnnotations[pieces[0]] = pieces[1]\n\n\treturn valAnnotations\n\nimport numpy as np\nfrom PIL import Image\n\ndef load_images(path,num_classes):\n    #Load images\n    \n    print('Loading ' + str(num_classes) + ' classes')\n\n    X_train=np.zeros([num_classes*500,3,32,32],dtype='uint8')\n    y_train=np.zeros([num_classes*500], dtype='uint8')\n\n    trainPath=path+'/train'\n\n    print('loading training images...');\n\n    i=0\n    j

In [0]:
'''


THE BELOW BLOCK CONTAINS THE MODEL ARCHITECTURE



'''

'\n\n\nTHE BELOW BLOCK CONTAINS THE MODEL ARCHITECTURE\n\n\n\n'

In [0]:

from __future__ import division

import tensorflow as tf
import tensorflow.keras as keras
import six
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Activation,
    Dense,
    Flatten
)
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    AveragePooling2D,
    SeparableConv2D
)
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K


In [0]:
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

import numpy as np

from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np
import math
from tensorflow.keras.callbacks import ModelCheckpoint

In [0]:
#Defining an Unit
'''
def Unit(x,filters,pool=False,padding="same"):
    res = x
    if pool:
        x = MaxPooling2D(pool_size=(2, 2))(x)
        res = SeparableConv2D(filters=filters,kernel_size=[1,1],strides=(2,2),padding=padding, depthwise_initializer='he_uniform', depth_multiplier=3)(res)
    out = BatchNormalization()(x)
    out = Activation("relu")(out)
    out = SeparableConv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding=padding, depthwise_initializer='he_uniform', depth_multiplier=3)(out)

    out = BatchNormalization()(out)
    out = Activation("relu")(out)
    out = SeparableConv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding=padding, depthwise_initializer='he_uniform', depth_multiplier=3)(out)

    out = keras.layers.concatenate([res,out])

    return out

'''

def Unit(x,filters,pool=False,padding="same"):
    res = x
    if pool:
        x = MaxPooling2D(pool_size=(2, 2))(x)
        res = Conv2D(filters=filters,kernel_size=[1,1],strides=(2,2),padding=padding, kernel_initializer='he_uniform')(res)
    out = BatchNormalization()(x)
    out = Activation("relu")(out)
    out = Conv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding=padding, kernel_initializer='he_uniform')(out)

    out = BatchNormalization()(out)
    out = Activation("relu")(out)
    out = Conv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding=padding, kernel_initializer='he_uniform')(out)

    out = keras.layers.add([res,out])

    return out
 



''' 
#Temporary Unit for easy model.summary() visualization
def Unit(x,filters,pool=False,padding="same"):
    res = x
    out = Conv2D(filters=filters, kernel_size=[5, 5], strides=[1, 1], padding=padding, kernel_initializer='he_uniform')(x)
    out = keras.layers.concatenate([res,out])

    return out'''


' \n#Temporary Unit for easy model.summary() visualization\ndef Unit(x,filters,pool=False,padding="same"):\n    res = x\n    out = Conv2D(filters=filters, kernel_size=[5, 5], strides=[1, 1], padding=padding, kernel_initializer=\'he_uniform\')(x)\n    out = keras.layers.concatenate([res,out])\n\n    return out'

In [0]:
#Build the model
from tensorflow.keras.layers import GlobalAveragePooling2D


def Architecture(input_shape):
  
    #Image Size here: 64x64
    images = Input(input_shape, name='input_image')
    
    net = Conv2D(filters=32, kernel_size=[3, 3], strides=[1, 1], padding="valid", kernel_initializer='he_uniform')(images)    #RF:3
    block1 = Unit(net,32)      #RF:7
    
    net = BatchNormalization()(block1)
    net = Activation("relu")(net)
    net = Conv2D(filters=64, kernel_size=[3, 3], strides=[1, 1], padding="valid", kernel_initializer='he_uniform')(net)      #RF:9      
    block2 = Unit(net,64)    #RF:13
    
    net = BatchNormalization()(block2)
    net = Activation("relu")(net)    
    net = Conv2D(filters=128, kernel_size=[3, 3], strides=[1, 1], padding="valid", kernel_initializer='he_uniform')(net)         #RF:15   
    block3 = Unit(net,128)    #RF:19
    
    net = BatchNormalization()(block3)
    net = Activation("relu")(net) 
    block4 = Unit(net,256,pool=True)     #RF:(19x2)+4=42

    #Image Size from o/p of block4: 58/2= 29x29
    
    net = BatchNormalization()(block4)
    net = Activation("relu")(net)  
    net = Conv2D(filters=400, kernel_size=[3, 3], strides=[1, 1], padding="valid", kernel_initializer='he_uniform')(net)        #RF:44
    block5 = Unit(net,400)    #RF:48

    net = BatchNormalization()(block5)
    net = Activation("relu")(net)  
    net = Conv2D(filters=512, kernel_size=[3, 3], strides=[1, 1], padding="valid", kernel_initializer='he_uniform')(net)        #RF:50
    block6 = Unit(net,512)    #RF:54

    net = BatchNormalization()(block6)
    net = Activation("relu")(net)  
    net = Conv2D(filters=768, kernel_size=[3, 3], strides=[1, 1], padding="valid", kernel_initializer='he_uniform')(net)        #RF:56
    block7 = Unit(net,768)    #RF:    60
    
    net = BatchNormalization()(block7)
    net = Activation("relu")(net)  
    net = Conv2D(filters=1024, kernel_size=[3, 3], strides=[1, 1], padding="valid", kernel_initializer='he_uniform')(net)        #RF:62
    block8 = Unit(net,1024)    #RF:    66
    
    net = BatchNormalization()(block8)
    net = Activation("relu")(net) 
    net = Conv2D(filters=180, kernel_size=[1, 1], strides=[1, 1], padding="same", kernel_initializer='he_uniform')(net)        #RF:66
    block9 = Unit(net,180)     #RF:70

    net = BatchNormalization()(block9)
    net = Activation("relu")(net) 
    net = Conv2D(filters=200, kernel_size=[3, 3], strides=[1, 1], padding="valid", kernel_initializer='he_uniform')(net)        #RF:72
    block10 = Unit(net,200)     #RF:76
    
    net = BatchNormalization()(block10)
    net = Activation("relu")(net) 
    net = Conv2D(filters=200, kernel_size=[3, 3], strides=[1, 1], padding="valid", kernel_initializer='he_uniform')(net)        #RF:78
    block11 = Unit(net,200)     #RF:82
    
    net = BatchNormalization()(block11)
    net = Activation("relu")(net) 
    net = Conv2D(filters=200, kernel_size=[3, 3], strides=[1, 1], padding="valid", kernel_initializer='he_uniform')(net)        #RF:84
    block12 = Unit(net,200)     #RF:88  
        
    net = BatchNormalization()(block12)
    net = Activation("relu")(net) 
    net = Conv2D(filters=200, kernel_size=[3, 3], strides=[1, 1], padding="valid", kernel_initializer='he_uniform')(net)        #RF:90
    block13 = Unit(net,200)     #RF:94
    
    #Image Size from o/p of block13: 13x13
    
    net = BatchNormalization()(block13)
    net = Activation("relu")(net) 
    net = GlobalAveragePooling2D()(net) #Preferring to use GlobalAvgPooling rather than GlobalMaxPooling to flatten as 200
    net = Activation('softmax')(net)

    model = Model(inputs=images,outputs=net)

    return model



In [0]:
#define a common unit

input_shape = (64,64,3)
model = Architecture(input_shape)

NameError: ignored

In [0]:
#Input Details

nb_classes = 200
data_augmentation = True


# input image dimensions
img_rows_train, img_cols_train = 64,64
img_rows, img_cols = 64, 64
# The CIFAR10 images are RGB.
img_channels = 3



In [0]:
#Data Preprocessing

#Preprocessing Data from channel_first to channel_last format 

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows_train, img_cols_train)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else: #This should take effect here since we need are using channels_last format and need to change our dataset that way
    X_train = X_train.reshape(X_train.shape[0], img_rows_train, img_cols_train, 3)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

    
    
# The data, shuffled and split between train and test sets:
#(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Convert class vectors to binary class matrices.
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# subtract mean and normalize
mean_image_train = np.mean(X_train, axis=0)
mean_image = np.mean(X_test, axis=0)
X_train -= mean_image_train
X_test -= mean_image
'''
X_train /= 128.
X_test /= 128.
'''

X_train /= 255.
X_test /= 255.


In [0]:
#Compiling the model
'''
#learning_rate=0.01
#optimizer=tf.train.AdadeltaOptimizer(learning_rate = learning_rate)
model.compile(loss='categorical_crossentropy',
              optimizer='Adadelta',
              metrics=['accuracy'])
'''
from tensorflow.keras.optimizers import SGD
learning_rate=0.01
custom_sgd = SGD(lr=learning_rate, momentum=0.9, decay=0.001, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=custom_sgd,
              metrics=['accuracy'])
              
'''
from keras.optimizers import Adam
custom_adam = Adam(lr=learning_rate, decay=decay_rate)

model.compile(loss='categorical_crossentropy',
              optimizer=custom_adam,
              metrics=['accuracy'])
'''

"\nfrom keras.optimizers import Adam\ncustom_adam = Adam(lr=learning_rate, decay=decay_rate)\n\nmodel.compile(loss='categorical_crossentropy',\n              optimizer=custom_adam,\n              metrics=['accuracy'])\n"

In [0]:
#model = tf.contrib.tpu.keras_to_tpu_model(
    model_gpu,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(
            tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.48.7.122:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 1110376956630918530)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5641881391192314564)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 1544634117433078345)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, 

In [0]:
#Checking model summary

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 62, 62, 32)   896         input_image[0][0]                
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 62, 62, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 62, 62, 32)   0           batch_normalization_v1[0][0]     
__________________________________________________________________________________________________
conv2d_1 (

In [0]:
#CLR Class Module

#CLR 
from tensorflow.keras.callbacks import *


class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

        
        

In [0]:
#Callbacks
from tensorflow.keras.callbacks import ModelCheckpoint

early_stopper = EarlyStopping(min_delta=0.001, patience=10)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

# Creating checkpoint after every epoch using callback
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.h5"
filepath_drive="/content/gdrive/My Drive/Colab Notebooks/RohanVer/"+filepath
checkpoint = ModelCheckpoint(filepath_drive, monitor='val_acc', verbose=1, save_best_only=True, mode='max', save_weights_only=False) #Saving the entire model and not just weights .ie.model.save(filepath)

#CLR Callback
clr = CyclicLR(base_lr=0.001, max_lr=0.1, step_size=200.)

#Learning rate print at end for SGD/Adam
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback
import tensorflow
class SGDLearningRateTracker(Callback):
    def on_epoch_end(self, epoch, logs={}):
        optimizer = self.model.optimizer
        _lr = tensorflow.to_float(optimizer.lr, name='ToFloat')
        _decay = tensorflow.to_float(optimizer.decay, name='ToFloat')
        _iter = tensorflow.to_float(optimizer.iterations, name='ToFloat')
        lr = K.eval(_lr * (1. / (1. + _decay * _iter)))
        print(' - LR: {:.6f}\n'.format(lr))
        
class AdamLearningRateTracker(Callback):
    def on_epoch_end(self, epoch, logs={}):
        #beta_1=0.9, beta_2=0.999
        optimizer = self.model.optimizer
        if optimizer.decay>0:
            lr = K.eval(optimizer.lr * (1. / (1. + optimizer.decay * optimizer.iterations)))
        t = K.cast(optimizer.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(beta_2, t)) /(1. - K.pow(beta_1, t)))
        print('\nLR: {:.6f}\n'.format(lr_t))       
        
        
        
        
callbacks_clr = [clr, checkpoint, lr_reducer, early_stopper, SGDLearningRateTracker()]

callbacks = [checkpoint, lr_reducer, early_stopper, SGDLearningRateTracker()]


In [0]:
'''

THE BELOW BLOCKS ARE FOR TRAINING THE MODEL


'''

'\n\nTHE BELOW BLOCKS ARE FOR TRAINING THE MODEL\n\n\n'

In [0]:
#TPU needs fixed image size...so can;t train on 32x32
'''
#Resizing the images to 32x32
import scipy
new_shape = (32,32,3)
X_train_new = np.empty(shape=(X_train.shape[0],)+new_shape)
for idx in range(X_train.shape[0]):
    X_train_new[idx] = scipy.misc.imresize(X_train[idx], new_shape)
X_test_new = np.empty(shape=(X_test.shape[0],)+new_shape)
for idx in range(X_test.shape[0]):
    X_test_new[idx] = scipy.misc.imresize(X_test[idx], new_shape)
'''

'\n#Resizing the images to 32x32\nimport scipy\nnew_shape = (32,32,3)\nX_train_new = np.empty(shape=(X_train.shape[0],)+new_shape)\nfor idx in range(X_train.shape[0]):\n    X_train_new[idx] = scipy.misc.imresize(X_train[idx], new_shape)\nX_test_new = np.empty(shape=(X_test.shape[0],)+new_shape)\nfor idx in range(X_test.shape[0]):\n    X_test_new[idx] = scipy.misc.imresize(X_test[idx], new_shape)\n'

In [0]:
from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips=None
    iaa.Dropout(0.02, name="Dropout"),
    iaa.OneOf([iaa.CoarsePepper(p=0.2,size_percent=0.02), iaa.CoarseSaltAndPepper(p=0.2,size_percent=0.05)]),
    sometimes(iaa.Affine(
            scale={"x": (0.5, 0.8), "y": (0.5, 0.8)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
            rotate=(-45, 45), # rotate by -45 to +45 degrees
            shear=(-16, 16), # shear by -16 to +16 degrees
            cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )),
], random_order=False) # apply augmenters in random order


ig = ImageDataGenerator(preprocessing_function=seq.augment_image)  # pass this as the preprocessing function

#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=64),
                        steps_per_epoch=400, #X_train.shape[0]//64,
                        epochs=1,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=[checkpoint])

INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(8,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(8, 64, 64, 3), dtype=tf.float32, name='input_image_10'), TensorSpec(shape=(8, 200), dtype=tf.float32, name='activation_39_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_image
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 41.15779376029968 secs
INFO:tensorflow:Setting weights on TPU model.
 445/1563 [=======>......................] - ETA: 10:53 - loss: 5.3109 - acc: 0.0058INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(4,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(4, 64, 64, 3), dtype=tf.float32, name='input_image_10'), TensorSpec(shape=(4, 200), dtype=tf.float32, name='activation_39_ta

In [0]:
from tensorflow.keras.models import load_model
model.save('/content/gdrive/My Drive/Colab Notebooks/RohanVer/model-cell-1-final.h5')

INFO:tensorflow:Copying TPU weights to the CPU


In [0]:
from tensorflow.keras.models import load_model
model.load_weights('/content/gdrive/My Drive/Colab Notebooks/RohanVer/first20epochs.h5')

In [0]:
batch_size=64

In [0]:
from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.0) # horizontal flips=None
], random_order=False) # apply augmenters in random order


ig = ImageDataGenerator(
        preprocessing_function=seq.augment_image, #pass this as the preprocessing function
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size, #X_train.shape[0]//64,
                        epochs=20,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=[checkpoint])

Epoch 1/20
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(16,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(16, 64, 64, 3), dtype=tf.float32, name='input_image_10'), TensorSpec(shape=(16, 200), dtype=tf.float32, name='activation_39_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_image
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 41.90829586982727 secs
781/782 [============================>.] - ETA: 0s - loss: 5.1468 - acc: 0.0194INFO:tensorflow:New input shapes; (re-)compiling: mode=eval (# of cores 8), [TensorSpec(shape=(16,), dtype=tf.int32, name='core_id_10'), TensorSpec(shape=(16, 64, 64, 3), dtype=tf.float32, name='input_image_10'), TensorSpec(shape=(16, 200), dtype=tf.float32, name='activation_39_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_image
INFO:tensorflow:

In [0]:
from tensorflow.keras.models import load_model
model.save('/content/gdrive/My Drive/Colab Notebooks/RohanVer/first20epochs.h5')

INFO:tensorflow:Copying TPU weights to the CPU


In [0]:
from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.0) # horizontal flips=None
], random_order=False) # apply augmenters in random order


ig = ImageDataGenerator(
        preprocessing_function=seq.augment_image, #pass this as the preprocessing function
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size, #X_train.shape[0]//64,
                        epochs=50,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=[checkpoint])

Epoch 1/50
10000/10000 [==============================] - 9s 897us/sample - loss: 3.7517 - acc: 0.2072

Epoch 00001: val_acc improved from 0.20150 to 0.20720, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-01-0.21.h5
INFO:tensorflow:Copying TPU weights to the CPU
782/782 [==============================] - 393s 503ms/step - loss: 3.8524 - acc: 0.1978 - val_loss: 3.7401 - val_acc: 0.2072
Epoch 2/50
10000/10000 [==============================] - 9s 908us/sample - loss: 3.7183 - acc: 0.2235

Epoch 00002: val_acc improved from 0.20720 to 0.22350, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-02-0.22.h5
INFO:tensorflow:Copying TPU weights to the CPU
782/782 [==============================] - 394s 504ms/step - loss: 3.8151 - acc: 0.2059 - val_loss: 3.7067 - val_acc: 0.2235
Epoch 3/50
10000/10000 [==============================] - 9s 902us/sample - loss: 3.8090 - acc: 0.2053

Epoch 00003: val_acc did not improve fro

In [0]:
from tensorflow.keras.models import load_model
model.save('/content/gdrive/My Drive/Colab Notebooks/RohanVer/first20_50epochs.h5')

In [0]:
from tensorflow.keras.models import load_model
model.load_weights('/content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-33-0.27.h5')

In [0]:
from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.0) # horizontal flips=None
], random_order=False) # apply augmenters in random order


ig = ImageDataGenerator(
        preprocessing_function=seq.augment_image, #pass this as the preprocessing function
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size, #X_train.shape[0]//64,
                        epochs=20,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=[checkpoint])

Epoch 1/20
10000/10000 [==============================] - 126s 13ms/sample - loss: 4.7589 - acc: 0.0437

Epoch 00001: val_acc improved from -inf to 0.04370, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-01-0.04.h5
1563/1563 [==============================] - 4253s 3s/step - loss: 4.8807 - acc: 0.0361 - val_loss: 4.7594 - val_acc: 0.0437
Epoch 2/20
10000/10000 [==============================] - 125s 13ms/sample - loss: 4.4108 - acc: 0.0797

Epoch 00002: val_acc improved from 0.04370 to 0.07970, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-02-0.08.h5
1563/1563 [==============================] - 4201s 3s/step - loss: 4.4837 - acc: 0.0746 - val_loss: 4.4085 - val_acc: 0.0797
Epoch 3/20
10000/10000 [==============================] - 126s 13ms/sample - loss: 4.0927 - acc: 0.1288

Epoch 00003: val_acc improved from 0.07970 to 0.12880, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-impro

In [0]:
from tensorflow.keras.models import load_model
model=load_model('/content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-10-0.26.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.0) # horizontal flips=None
], random_order=False) # apply augmenters in random order


ig = ImageDataGenerator(
        preprocessing_function=seq.augment_image, #pass this as the preprocessing function
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size, #X_train.shape[0]//64,
                        epochs=20,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=[checkpoint])

Epoch 1/20
10000/10000 [==============================] - 138s 14ms/sample - loss: 3.2923 - acc: 0.2644

Epoch 00001: val_acc improved from -inf to 0.26440, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-01-0.26.h5
1563/1563 [==============================] - 4582s 3s/step - loss: 3.1228 - acc: 0.2832 - val_loss: 3.2893 - val_acc: 0.2644
Epoch 2/20
 927/1563 [================>.............] - ETA: 29:37 - loss: 3.0335 - acc: 0.3001

In [0]:
from tensorflow.keras.models import load_model
model=load_model('/content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-01-0.26.h5')

OSError: ignored

In [0]:
batch_size=40

from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.0) # horizontal flips=None
], random_order=False) # apply augmenters in random order


ig = ImageDataGenerator(
        preprocessing_function=seq.augment_image, #pass this as the preprocessing function
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size, #X_train.shape[0]//64,
                        epochs=20,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=[checkpoint])

Epoch 1/20
10000/10000 [==============================] - 155s 16ms/sample - loss: 3.4150 - acc: 0.2527

Epoch 00001: val_acc improved from -inf to 0.25270, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-01-0.25.h5
2500/2500 [==============================] - 4915s 2s/step - loss: 3.2321 - acc: 0.2663 - val_loss: 3.4150 - val_acc: 0.2527
Epoch 2/20
10000/10000 [==============================] - 153s 15ms/sample - loss: 3.3030 - acc: 0.2668

Epoch 00002: val_acc improved from 0.25270 to 0.26680, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-02-0.27.h5
2500/2500 [==============================] - 4851s 2s/step - loss: 3.1459 - acc: 0.2827 - val_loss: 3.3030 - val_acc: 0.2668
Epoch 3/20
10000/10000 [==============================] - 152s 15ms/sample - loss: 3.2758 - acc: 0.2785

Epoch 00003: val_acc improved from 0.26680 to 0.27850, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-impro

In [0]:
from tensorflow.keras.models import load_model
model=load_model('/content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-05-0.31.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
batch_size=40

from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.0) # horizontal flips=None
], random_order=False) # apply augmenters in random order


ig = ImageDataGenerator(
        preprocessing_function=seq.augment_image, #pass this as the preprocessing function
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size, #X_train.shape[0]//64,
                        epochs=20,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=[checkpoint])

Epoch 1/20
10000/10000 [==============================] - 167s 17ms/sample - loss: 3.1250 - acc: 0.3115

Epoch 00001: val_acc improved from -inf to 0.31150, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-01-0.31.h5
2500/2500 [==============================] - 5231s 2s/step - loss: 2.8415 - acc: 0.3439 - val_loss: 3.1250 - val_acc: 0.3115
Epoch 2/20
10000/10000 [==============================] - 164s 16ms/sample - loss: 3.0498 - acc: 0.3262

Epoch 00002: val_acc improved from 0.31150 to 0.32620, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-02-0.33.h5
2500/2500 [==============================] - 5182s 2s/step - loss: 2.7717 - acc: 0.3554 - val_loss: 3.0498 - val_acc: 0.3262
Epoch 3/20
10000/10000 [==============================] - 166s 17ms/sample - loss: 3.1967 - acc: 0.3138

Epoch 00003: val_acc did not improve from 0.32620
2500/2500 [==============================] - 5162s 2s/step - loss: 2.6960 - acc: 0.3

In [0]:
from tensorflow.keras.models import load_model
model=load_model('/content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-07-0.36.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
batch_size=40

from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.0) # horizontal flips=None
], random_order=False) # apply augmenters in random order


ig = ImageDataGenerator(
        preprocessing_function=seq.augment_image, #pass this as the preprocessing function
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size, #X_train.shape[0]//64,
                        epochs=20,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=[checkpoint])

Epoch 1/20
10000/10000 [==============================] - 148s 15ms/sample - loss: 3.0665 - acc: 0.3300

Epoch 00001: val_acc improved from -inf to 0.33000, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-01-0.33.h5
2500/2500 [==============================] - 4876s 2s/step - loss: 3.1149 - acc: 0.3138 - val_loss: 3.0665 - val_acc: 0.3300
Epoch 2/20
10000/10000 [==============================] - 146s 15ms/sample - loss: 2.9053 - acc: 0.3603

Epoch 00002: val_acc improved from 0.33000 to 0.36030, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-02-0.36.h5
2500/2500 [==============================] - 4816s 2s/step - loss: 2.5656 - acc: 0.4070 - val_loss: 2.9053 - val_acc: 0.3603
Epoch 3/20
10000/10000 [==============================] - 146s 15ms/sample - loss: 3.0127 - acc: 0.3511

Epoch 00003: val_acc did not improve from 0.36030
2500/2500 [==============================] - 4828s 2s/step - loss: 2.4482 - acc: 0.4

In [0]:
from tensorflow.keras.models import load_model
model=load_model('/content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-06-0.38.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
batch_size=40

from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.0) # horizontal flips=None
], random_order=False) # apply augmenters in random order


ig = ImageDataGenerator(
        preprocessing_function=seq.augment_image, #pass this as the preprocessing function
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size, #X_train.shape[0]//64,
                        epochs=20,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=[checkpoint])

Epoch 1/20
10000/10000 [==============================] - 150s 15ms/sample - loss: 2.9489 - acc: 0.3762

Epoch 00001: val_acc improved from -inf to 0.37620, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-01-0.38.h5
2500/2500 [==============================] - 4959s 2s/step - loss: 2.1449 - acc: 0.4913 - val_loss: 2.9489 - val_acc: 0.3762
Epoch 2/20
10000/10000 [==============================] - 149s 15ms/sample - loss: 3.0839 - acc: 0.3675

Epoch 00002: val_acc did not improve from 0.37620
2500/2500 [==============================] - 4900s 2s/step - loss: 2.0820 - acc: 0.5019 - val_loss: 3.0839 - val_acc: 0.3675
Epoch 3/20
10000/10000 [==============================] - 154s 15ms/sample - loss: 2.9385 - acc: 0.3840

Epoch 00003: val_acc improved from 0.37620 to 0.38400, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-03-0.38.h5
2500/2500 [==============================] - 4957s 2s/step - loss: 2.0192 - acc: 0.5

In [0]:
#del model
#from tensorflow.keras.models import load_model
model.load_weights('/content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-06-0.40.h5')

In [0]:
batch_size=40

from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.0) # horizontal flips=None
], random_order=False) # apply augmenters in random order


ig = ImageDataGenerator(
        preprocessing_function=seq.augment_image, #pass this as the preprocessing function
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size, #X_train.shape[0]//64,
                        epochs=20,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=callbacks_clr)

Epoch 1/20
10000/10000 [==============================] - 151s 15ms/sample - loss: 2.8421 - acc: 0.3882

Epoch 00001: val_acc improved from -inf to 0.38820, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-01-0.39.h5
Instructions for updating:
Use tf.cast instead.
 - LR: 0.014429

2500/2500 [==============================] - 4695s 2s/step - loss: 2.9656 - acc: 0.3618 - val_loss: 2.8421 - val_acc: 0.3882
Epoch 2/20
10000/10000 [==============================] - 151s 15ms/sample - loss: 2.8930 - acc: 0.4004

Epoch 00002: val_acc improved from 0.38820 to 0.40040, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-02-0.40.h5
 - LR: 0.016667

2500/2500 [==============================] - 4667s 2s/step - loss: 1.7177 - acc: 0.5928 - val_loss: 2.8930 - val_acc: 0.4004
Epoch 3/20
10000/10000 [==============================] - 151s 15ms/sample - loss: 2.8781 - acc: 0.4099

Epoch 00003: val_acc improved from 0.40040 to 0.4099

In [0]:
from tensorflow.keras.models import load_model
model=load_model('/content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-08-0.42.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
batch_size=40

from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.0) # horizontal flips=None
], random_order=False) # apply augmenters in random order


ig = ImageDataGenerator(
        preprocessing_function=seq.augment_image, #pass this as the preprocessing function
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size, #X_train.shape[0]//64,
                        epochs=20,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=callbacks_clr)

Epoch 1/20
10000/10000 [==============================] - 172s 17ms/sample - loss: 4.9835 - acc: 0.0333

Epoch 00001: val_acc improved from -inf to 0.03330, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-01-0.03.h5
Instructions for updating:
Use tf.cast instead.
 - LR: 0.002149

2500/2500 [==============================] - 5231s 2s/step - loss: 5.2573 - acc: 0.0137 - val_loss: 4.9835 - val_acc: 0.0333
Epoch 2/20
10000/10000 [==============================] - 172s 17ms/sample - loss: 4.9835 - acc: 0.0333

Epoch 00001: val_acc improved from -inf to 0.03330, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-01-0.03.h5
Instructions for updating:
Use tf.cast instead.
 - LR: 0.002149

2500/2500 [==============================] - 5231s 2s/step - loss: 5.2573 - acc: 0.0137 - val_loss: 4.9835 - val_acc: 0.0333
Epoch 2/20
10000/10000 [==============================] - 171s 17ms/sample - loss: 4.2742 - acc: 0.1469

Epoch 0

In [0]:
from tensorflow.keras.models import load_model
model=load_model('/content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-07-0.38.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
batch_size=40

from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.0) # horizontal flips=None
], random_order=False) # apply augmenters in random order


ig = ImageDataGenerator(
        preprocessing_function=seq.augment_image, #pass this as the preprocessing function
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size, #X_train.shape[0]//64,
                        epochs=20,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=callbacks_clr)

Epoch 1/20
10000/10000 [==============================] - 153s 15ms/sample - loss: 5.0711 - acc: 0.0224

Epoch 00001: val_acc improved from -inf to 0.02240, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-01-0.02.h5
Instructions for updating:
Use tf.cast instead.
 - LR: 0.001232

2500/2500 [==============================] - 4784s 2s/step - loss: 5.2955 - acc: 0.0103 - val_loss: 5.0711 - val_acc: 0.0224
Epoch 2/20
10000/10000 [==============================] - 155s 15ms/sample - loss: 4.5920 - acc: 0.0992

Epoch 00002: val_acc improved from 0.02240 to 0.09920, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-02-0.10.h5
 - LR: 0.002299

2500/2500 [==============================] - 4771s 2s/step - loss: 4.6513 - acc: 0.0719 - val_loss: 4.5920 - val_acc: 0.0992
Epoch 3/20
10000/10000 [==============================] - 154s 15ms/sample - loss: 4.0033 - acc: 0.2053

Epoch 00003: val_acc improved from 0.09920 to 0.2053

In [0]:
from tensorflow.keras.models import load_model
model=load_model('/content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-08-0.38.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
batch_size=40

from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips=None
    iaa.Dropout(0.02, name="Dropout")
], random_order=False) # apply augmenters in random order


ig = ImageDataGenerator(
        preprocessing_function=seq.augment_image, #pass this as the preprocessing function
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=20,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=[checkpoint])

Epoch 1/20
10000/10000 [==============================] - 167s 17ms/sample - loss: 3.2080 - acc: 0.3660

Epoch 00001: val_acc improved from -inf to 0.36600, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-01-0.37.h5
2500/2500 [==============================] - 5348s 2s/step - loss: 2.3137 - acc: 0.4812 - val_loss: 3.2080 - val_acc: 0.3660
Epoch 2/20
10000/10000 [==============================] - 164s 16ms/sample - loss: 3.2302 - acc: 0.3723

Epoch 00002: val_acc improved from 0.36600 to 0.37230, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-02-0.37.h5
2500/2500 [==============================] - 5252s 2s/step - loss: 1.5819 - acc: 0.6192 - val_loss: 3.2302 - val_acc: 0.3723
Epoch 3/20
10000/10000 [==============================] - 166s 17ms/sample - loss: 3.2699 - acc: 0.3819

Epoch 00003: val_acc improved from 0.37230 to 0.38190, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-impro

In [0]:
from tensorflow.keras.models import load_model
model=load_model('/content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-05-0.38.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
batch_size=40

from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.0) # horizontal flips=None
], random_order=False) # apply augmenters in random order


ig = ImageDataGenerator(
        preprocessing_function=seq.augment_image, #pass this as the preprocessing function
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size, #X_train.shape[0]//64,
                        epochs=20,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=[checkpoint])

10000/10000 [==============================] - 158s 16ms/sample - loss: 3.4746 - acc: 0.3742

Epoch 00002: val_acc did not improve from 0.37550
2500/2500 [==============================] - 5158s 2s/step - loss: 1.2285 - acc: 0.7019 - val_loss: 3.4746 - val_acc: 0.3742

Epoch 00002: val_acc did not improve from 0.37550
2500/2500 [==============================] - 5158s 2s/step - loss: 1.2285 - acc: 0.7019 - val_loss: 3.4746 - val_acc: 0.3742
Epoch 3/20
Epoch 3/20
10000/10000 [==============================] - 156s 16ms/sample - loss: 3.6088 - acc: 0.3641

Epoch 00003: val_acc did not improve from 0.37550
2500/2500 [==============================] - 5158s 2s/step - loss: 1.1296 - acc: 0.7222 - val_loss: 3.6088 - val_acc: 0.3641

Epoch 00003: val_acc did not improve from 0.37550
2500/2500 [==============================] - 5158s 2s/step - loss: 1.1296 - acc: 0.7222 - val_loss: 3.6088 - val_acc: 0.3641
Epoch 4/20
Epoch 4/20
10000/10000 [==============================] - 159s 16ms/sample - 

KeyboardInterrupt: ignored

KeyboardInterrupt: ignored

In [0]:
from tensorflow.keras.models import load_model
model=load_model('/content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-05-0.04.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
batch_size=40

from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips=None
    iaa.Dropout(0.02, name="Dropout"),
    iaa.OneOf([iaa.CoarsePepper(p=0.2,size_percent=0.02), iaa.CoarseSaltAndPepper(p=0.2,size_percent=0.05)])
], random_order=False) # apply augmenters in random order


ig = ImageDataGenerator(
        preprocessing_function=seq.augment_image, #pass this as the preprocessing function
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=20,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=[checkpoint])

Epoch 1/20
10000/10000 [==============================] - 153s 15ms/sample - loss: 5.6890 - acc: 0.0551

Epoch 00001: val_acc improved from -inf to 0.05510, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-01-0.06.h5
2500/2500 [==============================] - 4997s 2s/step - loss: 3.4759 - acc: 0.2462 - val_loss: 5.6890 - val_acc: 0.0551
Epoch 2/20
10000/10000 [==============================] - 150s 15ms/sample - loss: 8.1825 - acc: 0.0064

Epoch 00002: val_acc did not improve from 0.05510
2500/2500 [==============================] - 4940s 2s/step - loss: 3.0678 - acc: 0.3197 - val_loss: 8.1825 - val_acc: 0.0064
Epoch 3/20
10000/10000 [==============================] - 151s 15ms/sample - loss: 5.5091 - acc: 0.0936

Epoch 00003: val_acc improved from 0.05510 to 0.09360, saving model to /content/gdrive/My Drive/Colab Notebooks/RohanVer/weights-improvement-03-0.09.h5
2500/2500 [==============================] - 4948s 2s/step - loss: 2.6657 - acc: 0.3

In [0]:
from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips=None
    iaa.Dropout(0.02, name="Dropout"),
    iaa.OneOf([iaa.CoarsePepper(p=0.2,size_percent=0.02), iaa.CoarseSaltAndPepper(p=0.2,size_percent=0.05)]),
    sometimes(iaa.Affine(
            scale={"x": (0.5, 0.8), "y": (0.5, 0.8)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
            rotate=(-45, 45), # rotate by -45 to +45 degrees
            shear=(-16, 16), # shear by -16 to +16 degrees
            cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )),
], random_order=True) # apply augmenters in random order


ig = ImageDataGenerator(preprocessing_function=seq.augment_image)  # pass this as the preprocessing function

#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=25,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=[checkpoint])

Epoch 1/25
10000/10000 [==============================] - 9s 899us/sample - loss: 5.3190 - acc: 0.0050

Epoch 00001: val_acc did not improve from 0.26570
782/782 [==============================] - 372s 476ms/step - loss: 5.3133 - acc: 0.0069 - val_loss: 5.3192 - val_acc: 0.0050
Epoch 2/25
10000/10000 [==============================] - 9s 894us/sample - loss: 5.7206 - acc: 0.0083

Epoch 00002: val_acc did not improve from 0.26570
782/782 [==============================] - 371s 475ms/step - loss: 5.2349 - acc: 0.0127 - val_loss: 5.7210 - val_acc: 0.0083
Epoch 3/25
10000/10000 [==============================] - 9s 878us/sample - loss: 5.7568 - acc: 0.0160

Epoch 00003: val_acc did not improve from 0.26570
782/782 [==============================] - 372s 475ms/step - loss: 5.1440 - acc: 0.0217 - val_loss: 5.7543 - val_acc: 0.0160
Epoch 4/25
10000/10000 [==============================] - 9s 870us/sample - loss: 5.7280 - acc: 0.0232

Epoch 00004: val_acc did not improve from 0.26570
782/782 [

In [0]:
from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.25, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips=None
    iaa.Dropout(0.02, name="Dropout"),
    iaa.OneOf([iaa.CoarsePepper(p=0.2,size_percent=0.02), iaa.CoarseSaltAndPepper(p=0.2,size_percent=0.05)]),
    sometimes(iaa.Affine(
            scale={"x": (0.5, 0.8), "y": (0.5, 0.8)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
            rotate=(-30, 30), # rotate by -45 to +45 degrees
            shear=(-16, 16), # shear by -16 to +16 degrees
            cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )),
], random_order=False) # apply augmenters in random order


ig = ImageDataGenerator(preprocessing_function=seq.augment_image)  # pass this as the preprocessing function

#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=25,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=[checkpoint])

Epoch 1/25
10000/10000 [==============================] - 9s 871us/sample - loss: 4.2139 - acc: 0.1723

Epoch 00001: val_acc did not improve from 0.26570
782/782 [==============================] - 371s 474ms/step - loss: 4.2166 - acc: 0.1401 - val_loss: 4.2123 - val_acc: 0.1723
Epoch 2/25
10000/10000 [==============================] - 9s 878us/sample - loss: 4.1707 - acc: 0.1801

Epoch 00002: val_acc did not improve from 0.26570
782/782 [==============================] - 370s 474ms/step - loss: 4.1875 - acc: 0.1447 - val_loss: 4.1682 - val_acc: 0.1801
Epoch 3/25
10000/10000 [==============================] - 9s 875us/sample - loss: 4.1557 - acc: 0.1830

Epoch 00003: val_acc did not improve from 0.26570
782/782 [==============================] - 370s 474ms/step - loss: 4.1581 - acc: 0.1506 - val_loss: 4.1562 - val_acc: 0.1830
Epoch 4/25
10000/10000 [==============================] - 9s 858us/sample - loss: 4.0823 - acc: 0.1912

Epoch 00004: val_acc did not improve from 0.26570
782/782 [

In [0]:
from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
import imgaug as ia

seq = iaa.Sequential([
    iaa.Fliplr(0.5) # horizontal flips=None
], random_order=False) # apply augmenters in random order


ig = ImageDataGenerator(
        preprocessing_function=seq.augment_image, #pass this as the preprocessing function
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

ig.fit(X_train)

#Train our model using ImgAug Augmentation
model.fit_generator(ig.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size, #X_train.shape[0]//64,
                        epochs=20,
                        verbose=1,     
                        validation_data=(X_test, Y_test),
                        callbacks=[checkpoint])

In [0]:
from tensorflow.keras.models import load_model
model.save('/content/gdrive/My Drive/Colab Notebooks/RohanVer/model-cell-2-final.h5')

In [0]:
score = model.evaluate(X_test, Y_test, verbose=0)
print(score)

In [0]:
#del model
from keras.models import load_model
model = load_model('/content/gdrive/My Drive/Colab Notebooks/RohanVer/model-cell-1-final1.h5')

In [0]:
from imgaug import augmenters as iaa

#seq = iaa.Sequential([...])  # list of desired augmentors

seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    iaa.Sometimes(0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    # Strengthen or weaken the contrast in each image.
    iaa.ContrastNormalization((0.75, 1.5)),
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
], random_order=True) # apply augmenters in random order


ig = ImageDataGenerator(preprocessing_function=seq.augment_image)  # pass this as the preprocessing function

#gen = ig.flow_from_directory(data_dir)  # nothing else changes with the generator

train_generator = ig.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(32, 32), color_mode='rgb', 
                                                    batch_size=64, class_mode='categorical', shuffle=True, seed=42)



valid_datagen = ImageDataGenerator()
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(32, 32),
                                                    color_mode='rgb', class_mode='categorical', batch_size=64, shuffle=True, seed=42)

model.fit_generator(train_generator, 
                    epochs=20, 
                    steps_per_epoch=200, 
                    validation_steps=200, 
                    validation_data=validation_generator, callbacks=callbacks)


Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
Epoch 1/20
 50/200 [======>.......................] - ETA: 1:06 - loss: 5.7797 - acc: 0.0097

KeyboardInterrupt: ignored

In [0]:
import imgaug as ia
from imgaug import augmenters as iaa

# Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
# e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

# Define our sequence of augmentation steps that will be applied to every image
# All augmenters with per_channel=0.5 will sample one value _per image_
# in 50% of all cases. In all other cases they will sample new values
# _per channel_.
seq = iaa.Sequential([
        # apply the following augmenters to most images
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.0), # vertically flip 20% of all images
        # crop images by -5% to 10% of their height/width
        sometimes(iaa.CropAndPad(
            percent=(-0.05, 0.1),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
        )),
        sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
            rotate=(-30, 30), # rotate by -45 to +45 degrees
            shear=(-16, 16), # shear by -16 to +16 degrees
            order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )),
        # execute 0 to 5 of the following (less important) augmenters per image
        # don't execute all of them, as that would often be way too strong
        iaa.SomeOf((0, 5),
            [
                iaa.OneOf([
                    iaa.GaussianBlur((0, 1.0)), # blur images with a sigma between 0 and 3.0
                    iaa.AverageBlur(k=(1, 3)), # blur image using local means with kernel sizes between 2 and 7
                    iaa.MedianBlur(k=(1, 3)), # blur image using local medians with kernel sizes between 2 and 7
                ]),
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
                iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
                
                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images
                iaa.OneOf([
                    iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
                    iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
                ]),
                iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                iaa.AddToHueAndSaturation((-20, 20)), # change hue and saturation
                # either change the brightness of the whole image (sometimes
                # per channel) or change the brightness of subareas
                iaa.OneOf([
                    iaa.Multiply((0.5, 1.5), per_channel=0.5),
                    iaa.FrequencyNoiseAlpha(
                        exponent=(-4, 0),
                        first=iaa.Multiply((0.5, 1.5), per_channel=True),
                        second=iaa.ContrastNormalization((0.5, 2.0))
                    )
                ]),
                iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
                iaa.Grayscale(alpha=(0.0, 1.0)),
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
                sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
            ],
            random_order=True
        )], random_order=True)

'''
),
        # execute 0 to 5 of the following (less important) augmenters per image
        # don't execute all of them, as that would often be way too strong
        iaa.SomeOf((0, 5),
            [
                iaa.OneOf([
                    iaa.GaussianBlur((0, 1.0)), # blur images with a sigma between 0 and 3.0
                    iaa.AverageBlur(k=(1, 3)), # blur image using local means with kernel sizes between 2 and 7
                    iaa.MedianBlur(k=(1, 3)), # blur image using local medians with kernel sizes between 2 and 7
                ]),
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
                iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
                
                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images
                iaa.OneOf([
                    iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
                    iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
                ]),
                iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                iaa.AddToHueAndSaturation((-20, 20)), # change hue and saturation
                # either change the brightness of the whole image (sometimes
                # per channel) or change the brightness of subareas
                iaa.OneOf([
                    iaa.Multiply((0.5, 1.5), per_channel=0.5),
                    iaa.FrequencyNoiseAlpha(
                        exponent=(-4, 0),
                        first=iaa.Multiply((0.5, 1.5), per_channel=True),
                        second=iaa.ContrastNormalization((0.5, 2.0))
                    )
                ]),
                iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
                iaa.Grayscale(alpha=(0.0, 1.0)),
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
                sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
            ],
            random_order=True
'''


"\n),\n        # execute 0 to 5 of the following (less important) augmenters per image\n        # don't execute all of them, as that would often be way too strong\n        iaa.SomeOf((0, 5),\n            [\n                iaa.OneOf([\n                    iaa.GaussianBlur((0, 1.0)), # blur images with a sigma between 0 and 3.0\n                    iaa.AverageBlur(k=(1, 3)), # blur image using local means with kernel sizes between 2 and 7\n                    iaa.MedianBlur(k=(1, 3)), # blur image using local medians with kernel sizes between 2 and 7\n                ]),\n                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images\n                iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images\n                \n                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images\n                iaa.OneOf([\n                    iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 

In [0]:

ig = ImageDataGenerator(preprocessing_function=seq.augment_image)  # pass this as the preprocessing function


train_generator = ig.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(64, 64), color_mode='rgb', 
                                                    batch_size=500, class_mode='categorical', shuffle=True, seed=42)



valid_datagen = ImageDataGenerator(rotation_range=0)
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64, 64),
                        color_mode='rgb', class_mode='categorical', batch_size=500, shuffle=True, seed=42)

model.fit_generator(train_generator, 
                    epochs=20, 
                    steps_per_epoch=200, 
                    validation_steps=200,  
                    validation_data=validation_generator, 
                    callbacks=[checkpoint])


Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
Epoch 1/20


AssertionError: ignored